In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import warnings
import logging

In [ ]:
base_path = "/sfs/gpfs/tardis/home/kcm7zp/brain_tumor_data"

tumor_types = ['glioma', 'meningioma', 'pituitary']
label_map = {'glioma': 1, 'meningioma': 2, 'pituitary': 3}

In [ ]:

data = []

for tumor in tumor_types:
    img_folder = os.path.join(base_path, tumor, "images")
    mask_folder = os.path.join(base_path, tumor, "masks")
    
    img_files = sorted([f for f in os.listdir(img_folder) if f.endswith(".png") and not f.startswith(".")])
    
    for img_file in tqdm(img_files, desc=f"Processing {tumor}"):
        base_id = img_file.replace(f"{tumor}_", "").replace(".png", "")
        mask_file = f"{tumor}_mask_{base_id}.png"
        
        img_path = os.path.join(img_folder, img_file)
        mask_path = os.path.join(mask_folder, mask_file)
        
        # Read in the image and mask 
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if img is None or mask is None:
            print(f"Failed to load: {img_path} or {mask_path}")
            continue

        # Resize to common dimensions
        img = cv2.resize(img, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        # Normalize to [0, 1]
        img = img / 255.0
        mask = mask / 255.0
        
        # Append to data list
        data.append({
            'image': img,
            'mask': mask,
            'label': label_map[tumor]
        })

print(f"\nTotal samples loaded: {len(data)}")

In [ ]:
# Convert to numpy arrays 
images = np.array([item['image'] for item in data])
labels = np.array([item['label'] for item in data])

# Add channel dimension for Tensorflow/Keras 
images = np.expand_dims(images, axis=-1) 

In [ ]:
# Train/Val/Test Split 
from sklearn.model_selection import train_test_split

# First split off test
X_trainval, X_test, y_trainval, y_test = train_test_split(images, labels, test_size=0.15, stratify=labels, random_state=3402)

# Then split train and val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.15, stratify=y_trainval, random_state=3402)